<a href="https://colab.research.google.com/github/tdlima/sistema_bancario-DIO/blob/main/sistema_bancario_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from abc import ABC, abstractclassmethod, abstractproperty
from datetime import datetime

In [3]:
class Cliente:
  def __init__(self,endr):
    self.endr = endr
    self.contas = []

  def realizar_transacao(self, conta, transacao):
    transacao.registrar(conta)

  def adcionar_conta(self, conta):
    self.contas.append(conta)

In [4]:
class PessoaFisica(Cliente):
  def __init__(self, nome, dt_nascimento, cpf, endr):
    super().__init__(endr)
    self.nome = nome
    self.dt_nascimento = dt_nascimento
    self.cpf = cpf

In [5]:
class Conta:
  def __init__(self, numero, cliente):
    self._saldo = 0
    self._numero = numero
    self._agencia = "0001"
    self._cliente = cliente
    self._historico = Historico()

  @classmethod
  def nova_conta(cls, cliente, numero):
    return cls(numero, cliente)

  @property
  def saldo(self):
    return self.saldo

  @property
  def numero(self):
    return self.numero

  @property
  def agencia(self):
    return self.agencia

  @property
  def cliente(self):
    return self.cliente

  @property
  def historico(self):
    return self.historico

  def sacar(self, valor):
    saldo = self.saldo
    saldo_excedido = valor > saldo

    if saldo_excedido:
      print("\nFalha na Operação! Saldo Insuficiente.")

    elif valor > 0:
      self.saldo -= valor
      print("\nSaque Realizado com Sucesso!")
      return True

    else:
      print("\nFalha na Operação! Valor Informado é Invalido.")
      return False

  def depositar(self, valor):
    if valor > 0:
      self.saldo += valor
      print("\nDeposito Realizado com Sucesso!")

    else:
      print("\nFalha na Operação! Valor Informado é Invalido.")
      return False

    return True

In [6]:
class ContaCorrente(Conta):
  def __init__(self, numero, cliente, limite=500, limite_saques=3):
    super().__init__(numero, cliente)
    self.limite = limite
    self.limite_saques = limite_saques

  def sacar(self, valor):
    numero_saques = len(
        [transacao for transacao in self.historico.
         transacoes if transacao["tipo"] == Saque.__name__]
    )

    limite_excedido = valor < self.limite
    saques_excedidos = numero_saques >= self.limite_saques

    if limite_excedido:
      print("\nFalha na Operação! O Valor a ser Sacado Excede o Limite.")

    elif saques_excedidos:
      print("\nFalha na Operação! Limite de Saques Diarios Excedidos.")

    else:
      return super().sacar(valor)

    return False

  def __str__(self):
    return f"""\
      Agência:\t{self.agencia}
      C/C:\t{self.numero}
      Titular:\t{self.cliente.nome}
    """

In [7]:
class Historico:
  def __init__(self):
    self._tramsacoes = []

  @property
  def transacoes(self):
    return self._transacoes

  def adicionar_transacao(self, transacao):
    self._transacoes.append({
        "tipo": transacao.__classe__.__name__,
        "valor": transacao.valor,
        "data": datetime.now().strftime("%d-%m%Y %H:%M:%s"),
    })

In [8]:
class Transacao(ABC):
  @property
  @abstractproperty
  def valor (self):
    pass

  @abstractclassmethod
  def registrar(self, conta):
    pass

In [9]:
class Saque(Transacao):
  def __init__(self, valor):
    self._valor = valor

  @property
  def valor(self):
    return self._valor

  def registrar(self, conta):
    transacao_ok = conta.sacar(self.valor)

    if transacao_ok:
      conta.historico.adicionar_transacao(self)

In [10]:
class Deposito(Transacao):
  def __init__(self, valor):
    self._valor = valor

  @property
  def valor(self):
    return self._valor

  def registrar(self, conta):
    transacao_ok = conta.depositar(self.valor)

    if transacao_ok:
      conta.historico.adicionar_transacao(self)

In [11]:
def menu():
  menu = input("\n### [1] Depósito | [2] Saque | [3] Extrato | [4] Cadastrar usuario | [5] Criar conta |\
  [6] Listar Contas [0] Sair ### 0\n\n Informe a opção desejada!\n\n")
  return menu

def filtrar_cliente(cpf, clientes):
    clientes_filtrados = [cliente for cliente in clientes if cliente.cpf == cpf]
    return clientes_filtrados[0] if clientes_filtrados else None


def recuperar_conta_cliente(cliente):
    if not cliente.contas:
        print("\n Cliente não possui conta!")
        return

    # FIXME: não permite cliente escolher a conta
    return cliente.contas[0]


def depositar(clientes):
    cpf = input("Informe o CPF do cliente: ")
    cliente = filtrar_cliente(cpf, clientes)

    if not cliente:
        print("\n Cliente não encontrado!")
        return

    valor = float(input("Informe o valor a ser depositado: "))
    transacao = Deposito(valor)

    conta = recuperar_conta_cliente(cliente)
    if not conta:
        return

    cliente.realizar_transacao(conta, transacao)


def sacar(clientes):
    cpf = input("Informe o CPF do cliente: ")
    cliente = filtrar_cliente(cpf, clientes)

    if not cliente:
        print("\nCliente não encontrado!")
        return

    valor = float(input("Informe o valor a ser sacado: "))
    transacao = Saque(valor)

    conta = recuperar_conta_cliente(cliente)
    if not conta:
        return

    cliente.realizar_transacao(conta, transacao)


def mostrar_extrato(clientes):
    cpf = input("Informe o CPF do cliente: ")
    cliente = filtrar_cliente(cpf, clientes)

    if not cliente:
        print("\n Cliente não encontrado!")
        return

    conta = recuperar_conta_cliente(cliente)
    if not conta:
        return

    print("\n================ EXTRATO ================")
    transacoes = conta.historico.transacoes

    extrato = ""
    if not transacoes:
        extrato = "Não foram realizadas movimentações."
    else:
        for transacao in transacoes:
            extrato += f"\n{transacao['tipo']}:\n\tR$ {transacao['valor']:.2f}"

    print(extrato)
    print(f"\nSaldo:\n\tR$ {conta.saldo:.2f}")
    print("==========================================")


def criar_cliente(clientes):
    cpf = input("Informe o CPF (somente número): ")
    cliente = filtrar_cliente(cpf, clientes)

    if cliente:
        print("\nCPF Já cadastrado")
        return

    nome = input("Informe o nome completo: ")
    data_nascimento = input("Informe a data de nascimento (dd-mm-aaaa): ")
    endereco = input("Informe o endereço (logradouro, nro - bairro - cidade/sigla estado): ")

    cliente = PessoaFisica(nome=nome, data_nascimento=data_nascimento, cpf=cpf, endereco=endereco)

    clientes.append(cliente)

    print("\n=== Cliente criado com sucesso! ===")


def criar_conta(numero_conta, clientes, contas):
    cpf = input("Informe o CPF do cliente: ")
    cliente = filtrar_cliente(cpf, clientes)

    if not cliente:
        print("\nCliente não encontrado, fluxo de criação de conta encerrado!")
        return

    conta = ContaCorrente.nova_conta(cliente=cliente, numero=numero_conta)
    contas.append(conta)
    cliente.contas.append(conta)

    print("\n=== Conta criada com sucesso! ===")


def listar_contas(contas):
    for conta in contas:
        print("=" * 100)

In [12]:
def main():

  clientes = []
  contas = []

  while True:

    opcao = menu()

    if opcao == "1":
      depositar(clientes)

    elif opcao == "2":
      sacar(clientes)

    elif opcao == "3":
      mostrar_extrato(clientes)

    elif opcao == "4":
      criar_cliente(clientes)

    elif opcao == "5":
      numero_conta = len(contas) + 1
      criar_conta(numero_conta, clientes, contas)

    elif opcao == "6":
      listar_contas(contas)

    elif opcao == "0":
      break

    else:
      print("\nOperação invalida. Selecione novamente a opção desejada!")


In [13]:
main()


### [1] Depósito | [2] Saque | [3] Extrato | [4] Cadastrar usuario | [5] Criar conta |  [6] Listar Contas [0] Sair ### 0

 Informe a opção desejada!

150

Operação invalida. Selecione novamente a opção desejada!

### [1] Depósito | [2] Saque | [3] Extrato | [4] Cadastrar usuario | [5] Criar conta |  [6] Listar Contas [0] Sair ### 0

 Informe a opção desejada!

1
Informe o CPF do cliente: 06314748623

 Cliente não encontrado!

### [1] Depósito | [2] Saque | [3] Extrato | [4] Cadastrar usuario | [5] Criar conta |  [6] Listar Contas [0] Sair ### 0

 Informe a opção desejada!

0
